## ETF Preprocessing

### Load pandas

In [22]:
import pandas as pd 

### Read in data & output dimensions

In [23]:
df = pd.read_csv('../../data/exchange-trade-funds-prices.csv')
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

Number of rows: 3866030
Number of columns: 8


### Remove empty columns

In [26]:
empty_cols2 = [col for col in df.columns if df[col].isnull().all()]
df.drop(empty_cols2,axis=1,inplace=True)

In [27]:
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

Number of rows: 3866030
Number of columns: 8


### Create transaction price variable

In [28]:
# approximation of price
df['transaction_price'] = df['adj_close']

### Create transaction volume variable

In [29]:
df['transaction_volume'] = df['volume']

### Create transaction value variable

In [30]:
df['transaction_value'] = df['transaction_price'] * df['transaction_volume']

### Convert date and subset data to 2000-2020

In [31]:
df['price_date'] = pd.to_datetime(df['price_date'])

In [32]:
start_date = '2000-01-01'
end_date = '2020-12-31'

In [33]:
new_df = df[(df['price_date'] >= start_date) & (df['price_date'] <= end_date)]

### Extract the year

In [34]:
new_df.loc[:, 'price_year'] = new_df['price_date'].dt.year

/var/folders/f6/ng6svcbs3x527pvn9dvrsjjr0000gn/T/ipykernel_51410/713853404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:, 'price_year'] = new_df['price_date'].dt.year


### Calculate the daily return - used for loss and risk

In [35]:
new_df.loc[:, 'daily_return'] = new_df.groupby('fund_symbol')['close'].pct_change()


/var/folders/f6/ng6svcbs3x527pvn9dvrsjjr0000gn/T/ipykernel_51410/847404955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:, 'daily_return'] = new_df.groupby('fund_symbol')['close'].pct_change()


### Aggregate by fund and year

In [38]:
aggregated_df = new_df.groupby(['fund_symbol', 'price_year']).agg({
    'open': 'mean',
    'high': 'mean',
    'low': 'mean',
    'close': 'mean',
    'adj_close': 'mean',
    'transaction_price': 'mean',
    'transaction_volume': 'mean',
    'transaction_value': 'mean',
    'daily_return': ['std', 'min']
}).reset_index()

### Rename columns

In [39]:
aggregated_df.columns = ['fund_symbol', 'price_year', 'avg_open', 'avg_high', 'avg_low', 'avg_close', 'avg_adj_close', 'avg_transaction_price', 'avg_transaction_volume', 'avg_transaction_value', 'yearly_risk', 'yearly_loss']

### Check dimensions

In [40]:
print("Number of rows:", aggregated_df.shape[0])
print("Number of columns:", aggregated_df.shape[1])

Number of rows: 14396
Number of columns: 12


### Preview data

In [41]:
aggregated_df.head()

,fund_symbol,price_year,avg_open,avg_high,avg_low,avg_close,avg_adj_close,avg_transaction_price,avg_transaction_volume,avg_transaction_value,yearly_risk,yearly_loss
0,AAA,2020,24.989500,24.996000,24.985000,24.988750,24.799375,24.799375,6360.000000,1.575589e+05,0.000700,-0.002405
1,AAAU,2018,12.182737,12.229579,12.148947,12.171579,12.171579,12.171579,103495.789474,1.252533e+06,0.006177,-0.012490
2,AAAU,2019,13.921230,13.968056,13.873294,13.920317,13.920317,13.920317,57093.650794,8.173461e+05,0.007424,-0.023196
3,AAAU,2020,17.696047,17.776917,17.577036,17.681818,17.681818,17.681818,411806.719368,7.529092e+06,0.012494,-0.054978
4,AADR,2010,27.727478,27.799913,27.633565,27.734522,26.044261,26.044261,5478.260870,1.424816e+05,0.009560,-0.034893


### Save dataset

In [42]:
aggregated_df.to_csv('../../data/cleaned-yearly-ETFs.csv', index=False)